In [1]:
import os
import csv
import requests
from duckduckgo_search import DDGS
from fastcore.all import *

# Define your image search function
def search_images(term, max_images=200):
    return L(DDGS().images(term, max_results=max_images)).itemgot('image')

# Create output folder
os.makedirs("images", exist_ok=True)

# Read CSV and get artwork names
image_names = []
with open("D-ViSA.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        image_names.append(row["artwork"].strip())

# Download images

#start from the letter "j"
for name in image_names:
    if name[0].lower() < "j":
        continue   
    search_term = name.replace("-", " ").replace(".jpg", "")
    print(f"Searching for: {search_term}")
    
    try:
        urls = search_images(search_term, max_images=5)
        if urls:
            image_url = urls[0]
            print(f"Found: {image_url}")
            response = requests.get(image_url, timeout=10)
            if response.status_code == 200:
                with open(f"images/{name}", "wb") as f:
                    f.write(response.content)
            else:
                print(f"Failed to download {name}: HTTP {response.status_code}")
        else:
            print(f"No image results for {name}")
    except Exception as e:
        print(f"Error processing {name}: {e}")


Searching for: jack bush_abstraction
Error processing jack-bush_abstraction.jpg: https://duckduckgo.com/i.js?l=wt-wt&o=json&q=jack+bush_abstraction&vqd=4-212010809101301216589648177796735156415&f=%2C%2C%2C%2C%2C&p=1 403 Ratelimit
Searching for: jack bush_blue pyramid 1947
Found: https://uploads7.wikiart.org/images/jack-bush/blue-pyramid-1947.jpg!Large.jpg
Searching for: jack bush_childhood 1955
Found: https://totallyhistory.com/wp-content/uploads/2012/08/childhood-1955-by-Jack-Bush.jpg
Searching for: jack bush_collage for christmas 1957
Found: https://uploads7.wikiart.org/images/jack-bush/collage-for-christmas-1957.jpg!Large.jpg
Searching for: jack bush_man and woman 1 1955
Found: https://m.media-amazon.com/images/I/61xpLnejkGL._SL1096_.jpg
Searching for: jack bush_may 1955
Found: https://uploads7.wikiart.org/images/jack-bush/may-1955.jpg!Large.jpg
Searching for: jack bush_november 19 oscar s death
Found: https://uploads5.wikiart.org/images/jack-bush/november-19-oscar-s-death.jpg
Searc

In [ ]:
import os
import pandas as pd

# Path to your folder with downloaded images
images_folder = r"processed_images"  # Replace with your actual folder path

# Read the original CSV
df = pd.read_csv("legacy/D-ViSA.csv")  # Replace with your actual CSV file path

# List of image filenames present in the folder
available_images = set(os.listdir(images_folder))

# Filter the DataFrame
filtered_df = df[df['artwork'].isin(available_images)]

# Save the filtered DataFrame to a new CSV
filtered_df.to_csv("filtered_artworks.csv", index=False)


df = pd.read_csv("filtered_artworks.csv")
available_images = set(os.listdir(images_folder))

#remove all images that are not in the filtered csv
for image in available_images:
    if image not in df['artwork'].values:
        os.remove(os.path.join(images_folder, image))
        print(f"Removed: {image}")
        
#save the filtered csv again
df.to_csv("filtered_artworks.csv", index=False)




FileNotFoundError: [Errno 2] No such file or directory: 'D-ViSA.csv'

In [15]:
import pandas as pd

df = pd.read_csv('filtered_artworks.csv')
print(df['final_emo'].value_counts())  # see class distribution

final_emo
fear           507
contentment    377
sadness        324
awe            311
excitement     284
amusement      256
disgust        205
anger           87
Name: count, dtype: int64


In [7]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['final_emo'], random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, stratify=test_df['final_emo'], random_state=42)

In [8]:
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)
test_df.to_csv('test.csv', index=False)


In [13]:
#process images to be 224x224 and save to new folder
import os

images_folder = r"images"  # Replace with your actual folder path
output_folder = r"processed_images"  # Replace with your desired output folder path
os.makedirs(output_folder, exist_ok=True)

from PIL import Image
from PIL.Image import UnidentifiedImageError
import pandas as pd

for index, row in df.iterrows():
    image_path = os.path.join(images_folder, row['artwork'])
    if os.path.exists(image_path):
        try:
            img = Image.open(image_path)
            img = img.resize((224, 224))
            if img.mode == 'RGBA':  # Convert RGBA to RGB
                img = img.convert('RGB')
            if img.mode != 'RGB':  # Ensure the image is in RGB mode
                img = img.convert('RGB')
            img.save(os.path.join(output_folder, row['artwork']), format='JPEG')
        except UnidentifiedImageError:
            print(f"Cannot identify image file: {image_path}")
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
    else:
        print(f"Image not found: {image_path}")

Cannot identify image file: images\friedel-dzubas_untitled(2).jpg
Cannot identify image file: images\hans-hofmann_ora-pro-nobis-1964.jpg
Cannot identify image file: images\jackson-pollock_man-with-knife-1940(1).jpg
Cannot identify image file: images\joan-mitchell_untitled-1957.jpg
Cannot identify image file: images\salvador-dali_arabs-the-death-of-raymond-lulle.jpg
